<a href="https://colab.research.google.com/github/leksandre/-crab-for-web2ipr_book_shop/blob/main/mocsTrainYOLOv3ForYoCol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLOv3 From Scratch with Darknet

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
! cd /content/drive/MyDrive/

In [79]:
! cd /content/drive/MyDrive/mocs_dataset/

In [80]:
%ls ./

backup/   data/      libdarknet.a    LICENSE.fuck  LICENSE.meta  Makefile  README.md  src/
cfg/      examples/  libdarknet.so*  LICENSE.gen   LICENSE.mit   obj/      results/
darknet*  include/   LICENSE         LICENSE.gpl   LICENSE.v1    python/   scripts/


In [81]:
!pwd

/content/YoCol/darknet


In [ ]:
# !unrar x -Y "/content/drive/MyDrive/mocs_dataset/instances_test.rar" "/content/instances_test/"

In [ ]:
# !unrar x -Y "/content/drive/MyDrive/mocs_dataset/instances_train.rar" "/content/instances_train/"

In [84]:
# !unrar x -Y "/content/drive/MyDrive/mocs_dataset/instances_test.rar" "/content/drive/MyDrive/mocs_dataset/instances_test/"

In [85]:
# !unrar x -Y "/content/drive/MyDrive/mocs_dataset/instances_train.rar" "/content/drive/MyDrive/mocs_dataset/instances_train/"

In [86]:
#not completed
# !unrar x -Y "/content/drive/MyDrive/mocs_dataset/instances_val.rar" "/content/drive/MyDrive/mocs_dataset/instances_val/"

In [87]:
!git clone https://github.com/Oskop/YoCol

Cloning into 'YoCol'...
remote: Enumerating objects: 1172, done.
remote: Total 1172 (delta 0), reused 0 (delta 0), pack-reused 1172
Receiving objects: 100% (1172/1172), 11.32 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (281/281), done.


## Setup Environment For Darknet

Since Google Colab has pre-installed CUDA 10, then we can skip CUDA configuration and go to next step

In [88]:
#Installing compilers
!apt install gcc-5 g++-5 -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package g++-5 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gcc-10-test-results gcc-9-test-results gcc-12-test-results gcc-11-test-results

E: Unable to locate package gcc-5
E: Package 'g++-5' has no installation candidate


In [89]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

ln: failed to create symbolic link '/usr/local/cuda/bin/gcc': File exists
ln: failed to create symbolic link '/usr/local/cuda/bin/g++': File exists


In [90]:
#Changing the variables to include OpenCV and GPU in the Makefile
! cd /content/YoCol/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
#!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile if you want to use cudnn

In [91]:
# Apparently we need to install this so that OpenCV can work without any issues
# when we are making the file
!apt-get install libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [92]:
# babakasd#break here

## Get Dataset

###Download and Extract COCO Dataset

Include COCO dataset that handled with get_coco_dataset.sh script so we don't need to convert label format from COCO format to YOLOv3 format. This step is an optional so you can skip if you think there's no need to including COCO dataset into training process. But if you not include COCO dataset, then you must use yolov3.weights from official site to continue the training process because the number of classes for this config is including COCO 80 classes

In [93]:
# %cd /content/YoCol/darknet
# !sh scripts/get_coco_dataset.sh

In [94]:
%cd /content/YoCol/darknet/coco
!paste <(awk "{pri!nt \"$PWD\"}" <5k.part) 5k.part | tr -d '\t' > 5k.txt
!paste <(awk "{print \"$PWD\"}" <trainvalno5k.part) trainvalno5k.part | tr -d '\t' > trainvalno5k.txt
%cd /content

[Errno 2] No such file or directory: '/content/YoCol/darknet/coco'
/content/YoCol/darknet
paste: 5k.part: No such file or directory
/bin/bash: line 1: 5k.part: No such file or directory
paste: trainvalno5k.part: No such file or directory
/bin/bash: line 1: trainvalno5k.part: No such file or directory
/content


### Labeling

We load the MATLAB annotations file first, then adding to a new dictionary that store both the Standford labels and COCO labels.

In [95]:
# load labels that i've already annotate to train and test
# standford images with YOLOv3 (contain 80 classes of COCO classes,
# 196 classes from Sandford classes are not include in this data.json file)
import json
cars_make_model_label = open('/content/drive/MyDrive/mocs_dataset/image_info_test.json', 'r')
standford_label = json.load(cars_make_model_label)

In [96]:
print(standford_label['categories'])
all_label = {}
all_names = []
all_images = {}

[{'supercategory': 'Construction', 'id': 1, 'name': 'Worker'}, {'supercategory': 'Construction', 'id': 2, 'name': 'Static crane'}, {'supercategory': 'Construction', 'id': 3, 'name': 'Hanging head'}, {'supercategory': 'Construction', 'id': 4, 'name': 'Crane'}, {'supercategory': 'Construction', 'id': 5, 'name': 'Roller'}, {'supercategory': 'Construction', 'id': 6, 'name': 'Bulldozer'}, {'supercategory': 'Construction', 'id': 7, 'name': 'Excavator'}, {'supercategory': 'Construction', 'id': 8, 'name': 'Truck'}, {'supercategory': 'Construction', 'id': 9, 'name': 'Loader'}, {'supercategory': 'Construction', 'id': 10, 'name': 'Pump truck'}, {'supercategory': 'Construction', 'id': 11, 'name': 'Concrete mixer'}, {'supercategory': 'Construction', 'id': 12, 'name': 'Pile driving'}, {'supercategory': 'Construction', 'id': 13, 'name': 'Other vehicle'}]


In [97]:
#process image_info_test.json categories
# Initialize dictionary that will contain all label from both labels data
for categories in standford_label['categories']:
  print(categories['name'])
  all_names.append(categories['name'])
  all_label[categories['name']] = []
print(all_label)

Worker
Static crane
Hanging head
Crane
Roller
Bulldozer
Excavator
Truck
Loader
Pump truck
Concrete mixer
Pile driving
Other vehicle
{'Worker': [], 'Static crane': [], 'Hanging head': [], 'Crane': [], 'Roller': [], 'Bulldozer': [], 'Excavator': [], 'Truck': [], 'Loader': [], 'Pump truck': [], 'Concrete mixer': [], 'Pile driving': [], 'Other vehicle': []}


In [98]:
#process image_info_test.json images
for images in standford_label['images']:
  all_images[images['file_name']] = images
print(len(all_images))

18264


In [99]:
%cd /content
import scipy.io as scio
import os
import cv2
# matlab_train_label = scio.loadmat('devkit/cars_train_annos.mat')
# matlab_test_label = scio.loadmat('devkit/cars_test_annos_withlabels.mat')
YOLO_CLASS = 13
TRAIN_IMAGE_PATH = '/content/drive/MyDrive/mocs_dataset/instances_train/'
TEST_IMAGE_PATH = '/content/drive/MyDrive/mocs_dataset/instances_train/'

/content


In [100]:
# Changing test label file name from "file_names_test" list to adjust
# the key format from data.json
# for name in file_names_test:
#   for file_name in name:
#     if (int(file_name.split('.')[0])+8144) < 10000:
#       new_name = '0' + str(int(file_name.split('.')[0]) + 8144) + '.jpg'
#       name[0] = new_name
#       os.rename(TEST_IMAGE_PATH + file_name, TEST_IMAGE_PATH + new_name)
#     else:
#       new_name = str(int(file_name.split('.')[0]) + 8144) + '.jpg'
#       name[0] = new_name
#       os.rename(TEST_IMAGE_PATH + file_name, TEST_IMAGE_PATH + new_name)

In [101]:
cars_make_model_label = open('/content/drive/MyDrive/mocs_dataset/instances_train.json', 'r')
train_label = json.load(cars_make_model_label)

In [102]:
cars_make_model_label = open('/content/drive/MyDrive/mocs_dataset/instances_val.json', 'r')
val_label = json.load(cars_make_model_label)

In [103]:
cars_make_model_label = open('/content/drive/MyDrive/mocs_dataset/image_info_test.json', 'r')
test_label = json.load(cars_make_model_label)

In [104]:
import cv2

In [105]:
def procStupidMocsYoloStruk(struck, prevListOfDicts = []):
  all_images = {}
  annotations = struck['annotations']
  images = struck['images']
  categories = struck['categories']
  for image in images:
    file_name = image['file_name']
    file_id = image['id']
    all_images[file_name] = []
    image = cv2.imread(TRAIN_IMAGE_PATH + file_name)
    height = image.shape[0]
    width = image.shape[1]
    filtered_list = list(filter(lambda x: x["image_id"] == file_id, annotations))
    for annotation in filtered_list:
      if 1:
      # if image.shape[1] >= 300:
          filtered_cats = list(filter(lambda x: x["id"] == annotation['category_id'], categories))
          filtered_cat = filtered_cat[0]
          obj_class = str(filtered_cat['name'])
          x1 = float(annotation['bbox'][0])
          x2 = float(annotation['bbox'][1])
          y1 = float(annotation['bbox'][2])
          y2 = float(annotation['bbox'][3])

    # if bbox[0] < 0:
    #   left = 0
    # else:
    #   left = bbox[0]
    # if bbox[1] < 0:
    #   top = 0
    # else:
    #   top = bbox[1]
    # if bbox[2] < 0:
    #   right = 1
    # else:
    #   right = bbox[2]
    # if bbox[3] < 0:
    #   bottom = 1
    # else:
    #   bottom = bbox[3]

          x_center = str(((x2-x1) / 2) / width)
          y_center = str(((y2-y1) / 2) / height)
          obj_width = str((x2-x1) / width)
          obj_height = str((y2-y1) / height)
          all_images[file_name].append(obj_class + ' ' + x_center + ' ' + y_center +
                                      ' ' + obj_width + ' ' + obj_height)

  prevListOfDicts[0] = all_images.update(prevListOfDicts[0] )
  return all_images

Then we move COCO Training and Validation images and its labels into Standford Images Directory. Labels data are moved to image directory because darknet will detect our label in same directory as images

In [106]:
# Moving COCO Images into Standford Images Directory
# !mv /content/YoCol/darknet/coco/images/train2014/* /content/cars_train
# !mv /content/YoCol/darknet/coco/images/val2014/* /content/casr_test

In [107]:
# # Moving COCO Labels into Standford Images Directory
# !mv /content/YoCol/darknet/coco/labels/train2014/* /content/cars_train
# !mv /content/YoCol/darknet/coco/labels/val2014/* /content/cars_test

Darknet need to a list file of image file path that will be passes to darknet for training and validation

In [108]:
# Make list file of train and test image file path
train_images_list = os.listdir(TRAIN_IMAGE_PATH)
test_images_list = os.listdir(TEST_IMAGE_PATH)
train_list_file = open('/content/train_list.txt', 'w')
test_list_file = open('/content/test_list.txt', 'w')
names_file = open('/content/names.txt', 'w')
for img_name in train_images_list:
  train_list_file.write(TRAIN_IMAGE_PATH + img_name + '\n')
for img_name2 in test_images_list:
  test_list_file.write(TEST_IMAGE_PATH + img_name2 + '\n')
for nameClass in all_names:
  names_file.write(nameClass + '\n')

In [109]:

# why they create huge txt files?
# Then we start labelling Standford Dataset based on labels data that stored in
# "all_label" dictionary

# for name in all_label:
#   if int(name[:-4]) < 8145 :
#     label_file = open(TRAIN_IMAGE_PATH + name[:-4] + '.txt', 'w')
#   else:
#     label_file = open(TEST_IMAGE_PATH + name[:-4] + '.txt', 'w')
#   for label in all_label[name]:
#     label_file.write(label + '\n')

##Setting before Training for Yolov3

Darknet need some configuration file befor training YOLO model that had ".data" extension. This file contains some configuration such as where darknet must take list file of training and validation, classes names that will use for YOLO, and path to store .weights file

In [110]:
!mkdir /content/bk/

mkdir: cannot create directory ‘/content/bk/’: File exists


In [111]:
!mkdir /content/weight
dat = """classes = 13
train = /content/train_list.txt
valid = /content/test_list.txt
names = /content/names.txt
backup = /content/bk/"""
with open('/content/darknet.data','w') as dark:
  dark.write(dat)

mkdir: cannot create directory ‘/content/weight’: File exists


####Compile Darknet

In [112]:
%cd /content/YoCol/darknet
!make
#Check if darknet is installed properly
!./darknet detector help

/content/YoCol/darknet
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -c ./src/gemm.c -o obj/gemm.o
Package opencv was not found in the pkg-config search path.
Perhaps you should add the directory containing `opencv.pc'
to the PKG_CONFIG_PATH environment variable
No package 'opencv' found
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdeprecated-declarations-Wdeprecated-declarations]8;;]
  232 |         cudaThreadSynchronize();
      |         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:95,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:1069:57: note: declared here
 1069 

##Start Training Yolov3

The training is start from scratch because we include both dataset and want to not only detect Standford Car classes name but also 80 COCO classes name

In [113]:
%cd /content/YoCol/darknet/
!./darknet detector train /content/darknet.data cfg/yolov3.cfg &> /dev/null

/content/YoCol/darknet


In [114]:
!cat /content/darknet.data

classes = 13
train = /content/train_list.txt
valid = /content/test_list.txt
names = /content/names.txt
backup = /content/bk/

In [115]:
!cat /content/names.txt

Worker
Static crane
Hanging head
Crane
Roller
Bulldozer
Excavator
Truck
Loader
Pump truck
Concrete mixer
Pile driving
Other vehicle


In [116]:
!head 10 /content/train_list.txt

head: cannot open '10' for reading: No such file or directory
==> /content/train_list.txt <==
/content/drive/MyDrive/mocs_dataset/instances_train/instances_train
/content/drive/MyDrive/mocs_dataset/instances_train/Readme.txt


In [117]:
!head 10 /content/test_list.txt

head: cannot open '10' for reading: No such file or directory
==> /content/test_list.txt <==
/content/drive/MyDrive/mocs_dataset/instances_train/instances_train
/content/drive/MyDrive/mocs_dataset/instances_train/Readme.txt


In [118]:
!cat cfg/yolov3.cfg

[net]
# Testing
# batch=1
# subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 552000
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutiona